In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

In [ ]:
import pandas as pd

from adapters.axiell.steps.loader import AXIELL_NAMESPACE
from adapters.axiell.helpers import build_window_store

window_store = build_window_store(use_rest_api_table=True)

print("\nWindow status table info:")
print(window_store.table)

DISPLAY_LIMIT = 20

In [ ]:
def window_status_frame(limit: int = 25, state: str | None = None) -> pd.DataFrame:

    """Return recent window status rows from the IcebergWindowStore."""
    rows = list(window_store.load_status_map().values())

    if not rows:
        return pd.DataFrame(
            columns=[
                "window_key",
                "window_start",
                "window_end",
                "state",
                "attempts",
                "last_error",
                "record_ids",
                "updated_at",
            ]
        )

    df = pd.DataFrame(rows)

    for column in ("window_start", "window_end", "updated_at"):
        if column in df.columns:
            df[column] = pd.to_datetime(df[column])

    df = df.sort_values("window_start", ascending=False)

    if state is not None:
        df = df[df["state"] == state]

    if limit is not None:
        df = df.head(limit)

    return df.reset_index(drop=True)

window_status_frame(limit=10)